In [1]:
""" 
json.dumps(): 将 Python 对象编码成 JSON 字符串。 json.dump(): 将 Python 对象编码成 JSON 字符串，并写入到文件中。json.loads(): 将已编码的 JSON 字符串解码为 Python 对象。json.load(): 读取文件中的 JSON 编码的数据，并解码为 Python 对象。
"""
import os
import json
import torch
torch.cuda.is_available()
def json_standard(input_object):

    # print(input_object,type(input_object))
    return json.dumps(input_object,ensure_ascii=False,indent=4)
def read_file(file_path:str):
    with open(file_path,"r",encoding="utf-8") as i_f:
        return json.load(i_f)
def write_file(file_path:str,obj,overwrite=False):
    if os.path.exists(file_path)==False or overwrite==True:
        with open(file_path,"w",encoding="utf-8") as o_f:
            json.dump(obj,o_f,ensure_ascii=False)
        print(f"写入{file_path}")
    else:
        print(f"{file_path}文件已经存在")
def check_file(file_path:str):
    """返回该路径下是否存在文件"""
    is_exist=os.path.exists(file_path)
    if is_exist:
        file_name=file_path.split("/")[-1]
        print(f"{file_name} exist")
    return is_exist

import torch
import json

from transformers import (AutoTokenizer)
model_id="/home/ckqsudo/code2024/0model/Mistral-7B-Instruct-v0.2"
tokenizer=AutoTokenizer.from_pretrained(model_id)
# 这里不设置 trust_remote_code也行
tokenizer.padding_size='right'#填充符的位置


In [292]:
cn_data=read_file("/home/ckqsudo/code2024/EMNLP2024/Chinese_data/cache/cn_data_正则处理错误.json")

In [ ]:
"请用20~100字为来访者生成用户画像"
"参考上文请告诉我咨询师所使用的技术，我作为咨询师应该如何指导患者？"
"请概括咨询师所使用的技术"

In [293]:
cn_data[0]

{'theme': '',
 'chat_stage': 'Regular Session',
 'is_same_qa': '',
 'is_same_case': '',
 'background': '',
 'pre_reasoning': '行为主义疗法\n行为主义疗法概述\n为主义疗法的核心目标在于消除来访者适应不良的行为方式，代之以更有建设性的行为方式。也就是说要找出导致来访者行为问题的思维方式，教以新的思维方法，从而改变他们原来的行为方式。\n常见的行为主义疗法技术有：系统脱敏法、实景暴露法、放松训练、强化、示范、社交技能训练、自我管理项目训练，专注与接纳技术、行为排练、指导以及其他多模式的技术。评估和诊断在确定治疗计划的阶段进行。其中，治疗师会采用“什么”、“怎样”以及“何时”这样的问题（但不会使用“为什么”这样的问题）。\n这种方法有其独特的优势：可以确定明确的、易于管理的和可测量的行为目标并可以将此作为治疗的焦点。在治疗开始时，个体的行为将被测量从而得到一个基线水平。\n因此，在后来的治疗过程中，治疗师就可以在给定的维度上将来访者的行为与其基线水平进行比较而得出治疗的进度。此外，在治疗过程中，评估和治疗是同时进行的，来访者将频繁地面对治疗师提出的这个问题：“我们现在做的事情是否可以帮助你达到你所预期的改变呢？”这样，来访者可以很容易地确定什么时候应该停止治疗。\n多模式行为主义疗法专家阿诺得·A·拉扎鲁斯博王对来访者的分析\n引言\n多模式疗法是一种系统的行为主义疗法，强调技术上的折衷主义，即根据来访者的特定问题选择不同的治疗手段。治疗师根据BASICI.D.领域（行为、情感、感知觉、意象、认知、人际关系、药物和生物因素）进行多模式评估，以确定具体问题。在行为层面，治疗师关注来访者的烦躁、避免眼神接触、睡眠不足等行为。在情感层面，治疗师关注焦虑、抑郁、恐慌等情感问题。在感知觉、意象、认知、人际关系和药物生物因素层面，治疗师也分别关注来访者的具体问题。治疗师通过多模式评估全面了解来访者的问题，为制定个性化的治疗计划提供了重要依据。和杰拉德·治疗师一样，我首先不会考虑采用诊断术语来对来访者进行定位。事实上，多模式的治疗师会将BASICI.D.的问题看做是“诊断”的结果。对我而言，进行一个具体详细的诊断并没有什么意义。很多DSM-IV-TR中的标签似

In [311]:
from openai import OpenAI
import os
import time
import copy
import json
client = OpenAI()
client.base_url='https://gptnb.keqichen.top/v1'
result=[]

  
token_sum=0
def get_price(total_token:int):
  return f"GPT-4:{token_sum/1000000*10}$ GPT-3:{token_sum/1000000*0.5}$"

for idx,topic in enumerate(cn_data):
    reasoning_list={ "pre_reasoning": cn_data[idx]["pre_reasoning"][-2000:],"post_reasoning": cn_data[idx]["post_reasoning"][:2000]}
    gpt_4="gpt-4-0613"
    gpt_3_5_model="gpt-3.5-turbo"
    for key,r in reasoning_list.items():
      input_obj={"input":r}
      
      if key =="pre_reasoning":
        example=[{"input":"借鉴了格式塔疗法的角色扮演技术：要求她像和她的父亲说话那样和我交流一一以第一人称来讲述那些她在电话上没有对她的父亲说的话。我选择鼓励她直接和我交谈一一看着我的眼睛一而不是像写报告那样交流，或“对着一把空椅子讲话”一通常，在长期的治疗过程中，我不太愿意采用这些技术，我更愿意选择进行支持性的倾听。也许因为受到只有半小时治疗时间的限制，这种更为“机动”的方式一一以第一人称和“交亲”对话一一一当时的确效果不错。如果我刻意地按照治疗的原理来进行操作一事实上我并没有这样做一一那么我就需要先设立一些治疗目标。其中一个就是要让来访者更加明晰自已的感受，并了解自已除了这些感受外还有哪些其他选择，了解自已是如何被这些感受所困的。我确信来访者对自已父亲的复杂情感远不止“非黑即白”这么简单。从长远的角度看，在角色扮演过程中表达自已不同方面的感受可以帮助她更好地理解不确定性、复杂性、自由等内容。\n这项技术是对计划外事件的一种创造性的改写，因为我希望在突发事件和治疗主题之间创建有意义的联系。例如，在第七次治疗时，我因时间问题急急忙忙地开始了治疗，来访者在其中还是更多地谈到了自已的父亲一一那个似乎从不会支持她的人，还有她的丈夫一一支持她来接受治疗的人，但是有个前提：约翰不希望治疗耽误他的晚餐，于·74·心理咨询与洽疗经典察例是我们进行了下面的对话："},{"output":"来访者可能是一个中年已婚女性，可能面临着与父亲和丈夫等关系复杂的挑战，渴望支持和理解。她可能在探索自我认知和情感复杂性方面有一定的意识。"}]
        messages=[
        {"role": "system", "content": f" 请基于给定文本完成任务：1.用20~100字为来访者生成用户画像，使用json格式返回修正后的结果,这是一个例子 " +json.dumps(example,ensure_ascii=False)},
        {"role": "user","content":json.dumps(input_obj,ensure_ascii=False)}
        ]

      completion = client.chat.completions.create(
          model=gpt_3_5_model,
          messages=messages,
      )
      token_sum+=completion.usage.total_tokens
      choice=completion.choices[0]
      time.sleep(0.5)
      # print(messages)
      print(idx,token_sum,get_price(token_sum),choice.message.content)
      cn_data[idx]['background']=choice.message.content
      write_file("/home/ckqsudo/code2024/EMNLP2024/Chinese_data/cache/gpt_OCR_cache.json",cn_data,True)
      
      
    # en_data[x]['topic_dialog']['dialog'][y]['emotional label']=choice.message.content


0 2559 GPT-4:0.02559$ GPT-3:0.0012795$ {"output": "来访者可能存在焦虑、避免眼神接触、睡眠不足等行为问题，以及沮丧、忧郁、惊恐等情感问题。在治疗中，她可能更注重具体的行为目标和行为改变，希望通过释恩谈练、行为预演和角色演练等技术来处理紧张和疾病，增强自我控制，并改变消极信念。治疗师可能会着重于个性化的治疗计划制定，以帮助来访者应对各种挑战。"}
写入/home/ckqsudo/code2024/EMNLP2024/Chinese_data/cache/gpt_OCR_cache.json
0 5126 GPT-4:0.05126$ GPT-3:0.002563$ {
    "output": "来访者可能是一个正在接受多模式行为主义治疗的个体，面临着焦虑、情感问题和自我认知方面的挑战。她可能正在寻求改变负面行为模式、缓解紧张和疾患、增强自我控制的帮助。治疗师通过多模式评估全面了解来访者的问题，制定个性化的治疗计划。在治疗中，她可能会通过释恩谈练、行为预演和角色演练等技术来处理问题，并逐步改变自我认知加强的信念。"
}
写入/home/ckqsudo/code2024/EMNLP2024/Chinese_data/cache/gpt_OCR_cache.json
1 6384 GPT-4:0.06384$ GPT-3:0.003192$ {"output": "来访者可能正在寻求情绪调节和认知重构方面的帮助，可能有处理负面情绪的需求。她可能也在关注人际关系和生活方式的改善，可能对婚姻治疗和家庭治疗感兴趣。整体而言，她似乎致力于个人成长和自我改善。"}
写入/home/ckqsudo/code2024/EMNLP2024/Chinese_data/cache/gpt_OCR_cache.json
1 7681 GPT-4:0.07681$ GPT-3:0.0038405$ {
  "output": "来访者可能在情绪调节、认知重构以及人际关系方面遇到挑战，可能正在寻求改善个人成长和生活方式。她可能接受治疗师的引导，愿意尝试多种策略来提升自我，同时也可能愿意参与婚姻治疗和家庭治疗以加强人际关系。综合来看，来访者可能是一个积极寻求改变和成长的人。"
}
写入/home/ckqsudo/code2024/EMNLP2024

In [406]:
cn_data=read_file("/home/ckqsudo/code2024/EMNLP2024/Chinese_data/cache/gpt_OCR_cache_backup.json")
def json_str_regular(j_str:str):
    if j_str.endswith(""",\n}"""):
        j_str=j_str[:-3]+"""\n}"""
    # print(j_str)
    return j_str
        
def regular_data(cur_data):
    del_keys=[
        "pre_reasoning_gpt",
        "post_reasoning_gpt"]
    json_dict={"pre_reasoning":"text","post_reasoning":"text","background":"output"}
    for idx,item in enumerate(cur_data):
        for del_key in del_keys:
            if del_key in item:
                del item[del_key]
        for key,o_key in json_dict.items():
            if item[key].startswith("{"):
                item[key]=json_str_regular(item[key])
                cur_data[idx][key]=json.loads(item[key])[o_key]
                # print(item[key],))
      
regular_data(cn_data)
write_file("/home/ckqsudo/code2024/EMNLP2024/Chinese_data/cache/gpt_OCR_cache_regular1.json",cn_data,True)

写入/home/ckqsudo/code2024/EMNLP2024/Chinese_data/cache/gpt_OCR_cache_regular1.json


In [344]:
# [item['background'][item['background'].find("。治疗师")+1:] for item in cn_data if "。治疗师" in item['background']]

In [404]:
for idx,item in enumerate(cn_data):
    bk=json.loads(item['background'])['output']
    stra=""
    if "。治疗师" in item['background']:
        stra=bk[bk.find("。治疗师")+1:]
        bk=bk[:bk.find("。治疗师")+1]
        # print(bk)
    elif "，治疗师" in item['background']:
        stra=bk[bk.find("，治疗师")+1:]
        bk=bk[:bk.find("，治疗师")]+"。"
    
    bk=bk.replace("来访者可能是","来访者是")
    print(bk,stra)
    if item["pre_reasoning"]=='' and "用户画像" in bk :
        # print("error",bk,cn_data[idx-1]['background'])
        bk=cn_data[idx-1]['background']
    cn_data[idx]['background']=bk
    cn_data[idx]['guide']=stra

    # print(":",bk)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [360]:
guide_prompt_dict=[]
for idx,item in enumerate(cn_data):
    if 'guide' in item and item['guide']!="":
        if ' observe ' in item['guide']:
            cn_data[idx]['guide']=cn_data[idx]['guide'].replace(' observe ',"观察")
        print("Guide: ",item['guide'].strip())
        print("########\n",item['pre_reasoning'],"\n\n\n")
        guide_prompt_dict.append({"example":item['pre_reasoning'],"guide":item['guide']})

Guide:  治疗师通过多模式评估全面了解来访者的问题，制定个性化的治疗计划。在治疗中，她可能会通过行为测量、行为预演和角色演练等技术来处理问题，并逐步改变自我认知加强的信念。
########
 {
    "text": "行为主义治疗\n行为主义治疗概述\n为主治疗法御法的核心目标在于消除来访者适应不良的行为方式，代之以更有建设性的行为方式。也就是说要找出导致来访者行为问题的思维方式，教以新的思维方式，从而改变他们原来的行为方式。\n常见的行为主义治疗技术有：系统离散法、实景暴露法、释恩谈练、强化、示范、社交技能练习、自我管理项目练习，专注于接纳技术、行为排练、指导以及其他多种方式的技术。评估和诊断在确定治疗计划的阶段进行。其中，治疗师会采用“什么”、“怎样”以及“何时”这样的问题（但不会使用“为什么”这样的问题）。\n这种方法有其独特的优势：可以确立明确的、易于管理的和可测量的行为目标并可以将此作为治疗的焦点。在治疗开始时，个体的行为将被测量，从而得到一个基准水平。\n因此，在后来的治疗过程中，治疗师就可以在给定的维度上将来访者的行为与其基准水平进行比较而得出治疗的进度。这外，评估和治疗是同时进行的，来访者将频繁地面对治疗师提出的这个问题：“我们现在做的事情是否可以帮助你达到你所预期的改变呢？”这样，来访者可以很容易确定什么时候应该停止治疗。\n多模式行为主义治疗专家阿诺·A·拉扎尔斯博士对来访者的分析\n引言\n多模式治疗是一种系统的行为主义治疗，强调技术上的折衷主义，即根据来访者的特定问题选择不同的治疗手段。治疗师根据BASICI.D.领域（行为、情绪、感知觉、意识、认知、人际关系、药物和生物因素）进行多模式评估，以确保具体问题。在行为层面，治疗师关注来访者的焦虑、避免眼神接触、睡眠不足等行为。在情感层面，治疗师关注沮丧、忧郁、惊恐等情感问题。在认知、感知、意识、人际关系和药物生物因素层面，治疗师也分别关注来访者的具体问题。治疗师通过多模式评估全面了解来访者的问题，为制定个性化的治疗计划提供了重要依据。和杰拉德·治疗师一样，我首先不会考虑采用诊断术语来对来访者进行定位。实际上，多模式的治疗师会将BASICI.D.的问题看做是“诊断”的结果。对我而言，进行一个具体的详细的诊断并没有什么意义。很多DSM-IV-TR中的标志似乎都和来访者的问题相符，比如：3

In [363]:
[item for item in guide_prompt_dict if len(str(item))<500]

[{'example': '治疗师的目标是帮助来访者意识到自己是生活的自由作者，面临着多种选择，并学会忍受不确定性。他希望来访者能够摆脱惊恐发作的无力感，保留一定的存在性焦虑，并在理性水平上提升自我觉察力，同时意识到更多的感受和行为选择。治疗师强调通过思考、感受和行动来实现这些目标，并期望来访者在情绪上有所变化，从而更好地面对不同情绪状态下的选择。治疗师采用存在主义疗法，强调在治疗过程中不存在特定的治疗技术，而是通过阅读自传、提供详细说明和建议进行治疗。他鼓励来访者详细解释自传中的问题，并提出可用的选择，同时在第一次会面结束时建议进行每周两次每次30分钟的治疗。治疗师的策略注重引导来访者自我意识和行为改变，以实现个体的自由和责任。',
  'guide': '治疗师强调通过反思、感知和行动引导她实现目标，希望她在情绪和行为上有所改变。建议定期治疗并重视个体自由和责任。'},
 {'example': '治疗师采用存在主义疗法，强调治疗关系的重要性，将治疗视为一种深刻的对话和共同的旅程。治疗师鼓励来访者质疑生活的意义，勇敢面对选择和焦虑，并自发地思考存在的意义。治疗目标不是消除症状，而是帮助来访者意识到自己的选择和决定对生活的影响，摆脱僵化的角色，追求自由和责任感。治疗过程旨在帮助来访者思索和了解自己的存在，澄清寻求帮助的目的和需要改变的方向。治疗师在处理来访者的过去经历时，鼓励她思考过去与现在的联系，并借鉴其他疗法的技术。在治疗过程中，治疗师提出问题，帮助来访者探索自己的生活和选择，解决焦虑和限制，打开心灵的“绝对”的伽锁，探索并实现选择。治疗过程中，来访者直面中年危机，认识到生活的价值观已经改变，表达了空虚感和恐惧。治疗师通过引导和鼓励，帮助来访者逐渐了解自己的内心世界，探索选择，并迈向未来。！帮助来访者发展出新的价值观在后来的一次治疗中，来访者从宗教说起了自己的问题：\n',
  'guide': '治疗师的存在主义疗法和鼓励引导帮助她勇敢面对挑战，追求自由和意义。'},
 {'example': '实验中有很多选择点。在这个实验中，治疗师可以关注来访者“被摆布”时的感受（实际上是他自已决定要前后移动的）或是“生气”的情绪。这类选择点经常出现，治疗方向可能被如下一些因素所决定：迄今为止何种成长边界从治疗关系的背景上浮现出来，以及被创造出来的突发事件有多安全（见第

In [407]:
guide_example=[{'input': '咨询需要接受辅导这此来访者势必会花上比较多的时间与心力，但是，SFBT也视这此来访者为具有合作潜力的光事人，贝要治疗师能辩识出他们真正想要的月标与资源，当然，如果来访者为儿童或青少年，治疗师也需要让教养与管理他们的大人了解这些孩了的优势以及所需学习之处，以能更成为适合他们的教导者。，3.运用重新建构的眼光，找寻来访者愿意试合作的讯号非白愿来谈来访者真的都是十分非自愿吗？其实每位来访者都有程度的不同，即使真的是极度非自愿的来访者，也可能会有愿意合作的尝试。面对非国愿来谈来访者一开始来的不合作态度，治疗师若能从中看到其愿意尝试含作的细微讯号，将会有助于关系的发展。例如：\n',
  'output': '治疗师若能观察到来访者愿意合作的微弱信号并鼓励来访者，将有助于建立更好的关系。'},
{'input': '虽然SFBT不会把情绪独立分割来处理，但是自然同理、整体接纳、肯定认同来访者的情绪，乃是SFBT治疗师在所有陪谈过程中持续的重要态度与责任。治疗师若不能一直同理、接纳来访者任何情绪反应，或让来访者觉得治疗师并不了解其情绪与经验，会谈将无法让来访者愿意转变为朝着追求个人目标或可题解决的方向前进，也将阻碍后续“解决式谈话”的推展与进行（许维素，2009&2014）。因此，对于来访者负向情绪的一般化或重新建构，是一种深沉接纳、尊重与理解，是要能展现常态化、正常化、去病理化的接纳与正向框架的态度，而不只是用来安慰、敷衍来访者的浅薄技巧，因而大大考验着治疗师是否能认识、包容处于各生活处境与生命阶段的来访者所会有的各种反应与心境（McNeilly，2000）。\n十案例对话与反思活动\n',
  'output': '治疗师的接纳态度和能力对于来访者的情绪重建和转变至关重要，这需要常态化、正常化的态度而非简单的安慰和敷衍。来访者可能正处于不同生活阶段，对生活各种反应和情绪有着多样化的体验。'},
   {'input': '治疗师做得很对，不用去等着看来访者会不会跟自已多谈一点室友的事，既然治疗师对此产生了兴趣，就不妨直接问。治疗师选择先问室友的事，而不是问论文，因为了解“室友问题”，或许能使人进一步理解来访者的初始主诉议题：在建立人际关系上遇到困难。\n在“来访者因自己公寓里凌乱而不悦”，与“她的童年记忆，以及当时她母亲抑郁发作后家里的状况”之间，治疗师可能会去尝试建立联系。尽管这之间可能会存在着某种联系，但在本节面询中，尚无证据下这个结论，所以，此刻治疗师没有理由不就其字面意涵来理解来访者。治疗师没有将来访者的思考强行纳入所谓心理动力取向的诠释模式，而是把握这个机会，去了解来访者在面对压力时的反应，了解她调适日常生活的能力。\n',
  'output': '治疗师选择先了解室友问题，因为这可能与来访者的初始主诉相关。治疗师不强行使用心理动力学，而是关注来访者面对压力的反应和应对日常生活的能力，为来访者提供温暖和支持。'},
   {'input': 'Clark先生被贴上了一个“抑郁症”的标签，转到你这里来就诊。这就意味着，见到来访者本人之前，你在心里已经对他有了某种预期。这种预期来自你过去经验中对抑郁来访者的了解，也来自你在资料里曾读到过的对这种综合征的印象。可是，你真的见到他后，尝试将他纳入这一诊断类别里时，不知道怎么地，你发现他跟你之前的预期合不上。接下来你必须要做的是，在和来访者继续谈话、继续检查时，觉察在你自己身上发生了什么。有时候你心底会冒出一个假设，你可以跟着这个假设往前走走看，去检验它。而其他时候，面询结束后，你对所发生的情况却依然会是一知半解的，就像你今天遇到的情况一样。不过，像你现在这样，承认自己搞不清状况，就远比周顾来访者的实际情况、削足适履地给他扣上诊断帽子要强得多。将来访者硬性归类，然后依据“诊断”制定出所谓的治疗策略，这样做只不过是在缓解治疗师的焦虑罢了，相比之下，承认自己搞不清状况要好得多了。\n',
  'output': '治疗师可能在与来访者继续对话和检查时意识到自己的偏见和预期，并可能采用一种更开放、不强行归类的方式来理解和支持来访者。'}
]


In [ ]:
for idx,item in enumerate(cn_data):
    cn_data[idx]['background']=json.loads(item['background'])['output']

In [369]:
write_file("/home/ckqsudo/code2024/EMNLP2024/Chinese_data/cache/gpt_OCR_cache_backup.json",cn_data,True)

写入/home/ckqsudo/code2024/EMNLP2024/Chinese_data/cache/gpt_OCR_cache_backup.json


In [378]:
idx


1

In [415]:
from openai import OpenAI
import os
import time
import copy
import json
client = OpenAI()
client.base_url='https://gptnb.keqichen.top/v1'
result=[]

  
token_sum=0
def get_price(total_token:int):
  return f"GPT-4:{token_sum/1000000*10}$ GPT-3:{token_sum/1000000*0.5}$"
cn_data=read_file("/home/ckqsudo/code2024/EMNLP2024/Chinese_data/cache/gpt_OCR_cache_regular1.json")
for idx,topic in enumerate(cn_data):
    reasoning_list={ "pre_reasoning": cn_data[idx]["pre_reasoning"][-1500:],"post_reasoning": cn_data[idx]["post_reasoning"][:1500]}
    gpt_4="gpt-4-0613"
    gpt_3_5_model="gpt-3.5-turbo"

    print(idx,cn_data[idx]['guide'])
    for key,r in reasoning_list.items():
      input_obj={"input":r}
      if key =="pre_reasoning":
        if 'guide' in cn_data[idx].keys() and len(cn_data[idx]['guide'])>3:
           print(idx,"has",cn_data[idx].keys(),cn_data[idx]['guide'])
           continue
        else:
            messages=[
            {"role": "system", "content": f" 请基于给定文本完成任务，下面是一些例子："+json.dumps(guide_example,ensure_ascii=False)+"1.结合输入文本用50~100字告诉我作为咨询师我应当使用什么方式帮助来访者，使用json格式返回修正后的结果: "},
            {"role": "user","content":json.dumps(input_obj,ensure_ascii=False)}
            ]
            completion = client.chat.completions.create(
                model=gpt_3_5_model,
                messages=messages,
            )
            token_sum+=completion.usage.total_tokens
            choice=completion.choices[0]
            time.sleep(0.5)
            # print(messages)
            print(idx,token_sum,get_price(token_sum),choice.message.content)
            response_str=choice.message.content
            cn_data[idx]['guide']=response_str
            write_file("/home/ckqsudo/code2024/EMNLP2024/Chinese_data/cache/gpt_OCR_cache_o.json",cn_data,True)

    

0 治疗师通过多模式评估全面了解来访者的问题，制定个性化的治疗计划。在治疗中，她可能会通过行为测量、行为预演和角色演练等技术来处理问题，并逐步改变自我认知加强的信念。
0 has dict_keys(['theme', 'chat_stage', 'is_same_qa', 'is_same_case', 'background', 'pre_reasoning', 'topic_dialog', 'post_reasoning', 'extra_tags', 'guide']) 治疗师通过多模式评估全面了解来访者的问题，制定个性化的治疗计划。在治疗中，她可能会通过行为测量、行为预演和角色演练等技术来处理问题，并逐步改变自我认知加强的信念。
1 
1 2487 GPT-4:0.02487$ GPT-3:0.0012435$ {"output": "治疗师可以采用情绪调节、认知重构、人际关系改善和生活方式调整等多种策略，如引导想象练习、使用失控法和认知重构、建议婚姻治疗和家庭治疗、制定饮食运动计划等，旨在全方位协助来访者实现个人成长和改善。"}
写入/home/ckqsudo/code2024/EMNLP2024/Chinese_data/cache/gpt_OCR_cache_o.json
2 
2 4802 GPT-4:0.04802$ GPT-3:0.002401$ 作为咨询师，您可以使用重新建构的方法帮助来访者。观察来访者愿意合作的微弱信号并鼓励来访者，将有助于建立更好的关系。给予常态化、正常化的态度，帮助来访者重建情绪和转变，让来访者感受到您的接纳和理解，从而支持他们朝着追求个人目标或解决问题的方向前进。
写入/home/ckqsudo/code2024/EMNLP2024/Chinese_data/cache/gpt_OCR_cache_o.json
3 
3 7654 GPT-4:0.07654$ GPT-3:0.003827$ {"output": "作为咨询师，您可以采取与来访者合作、提供鼓励和无偏见建议的方式来帮助来访者。通过帮助来访者挑战困扰她的观念和事件，引导她进行自我改变并拥有收获感。在促进联合治疗的过程中，需要关注来访者对改变的准备和她丈夫的配合程度。建议与来访者及其丈夫约翰进行开放对话，鼓励约翰参与治疗以达到共同成长的目的。"}
写入/h

In [23]:
cn_data=read_file("/home/ckqsudo/code2024/EMNLP2024/Chinese_data/cache/gpt_OCR_cache_o.json")
def json_str_regular(j_str:str):
    if j_str.endswith(""",\n}"""):
        j_str=j_str[:-3]+"""\n}"""
    # print(j_str)
    try:
        item=json.loads(j_str)
        item['output']
    except Exception as e:
        # 处理异常的代码
        errors=j_str.split("\"")
        print(errors)
        for i,item in enumerate(errors):
            if i>2 and errors[i-2]=='output' and errors[i-1]==': ':
                return json.dumps({"output":errors[i]}) 
    return j_str
def regular_data(cur_data):
    del_keys=[
        "pre_reasoning_gpt",
        "post_reasoning_gpt"]
    json_dict={"pre_reasoning":"text","post_reasoning":"text","background":"output","guide":"output"}
    for idx,item in enumerate(cur_data):
        for del_key in del_keys:
            if del_key in item:
                del item[del_key]
        for key,o_key in json_dict.items():
            if item[key].startswith("{"):
                item[key]=json_str_regular(item[key])
                if "message" in item[key] or 'error' in item[key]:
                    continue
                else:
                    cur_data[idx][key]=json.loads(item[key])[o_key]
regular_data(cn_data)


['{\n    ', 'message', ': ', '请输入需要修正的文本内容', '\n}']
['{', 'output', ': ', '作为咨询师，你可以观察来访者愿意合作的微弱信号并鼓励来访者，将有助于建立更好的关系。治疗师需要常态化、正常化的态度来接纳来访者的情绪重建和转变。对于来访者可能处于不同生活阶段的情绪体验，需要持续的接纳和理解。在与来访者交流时，采用开放、不强迫性的方式来支持和理解来访者，而不是简单地给他们贴上标签或诊断。', ', ', 'input', ': ', '我重述了她说过的话，在句尾用了疑问的语调，再一次地，用这种问句方式揭开了两种可能性，这件事可能发生也可能不会发生。\\n我想帮助她用任何可能的方式连接她和我的脸，我暗示她在保有或者失去我的脸这方面，她是主动的。这是我希望治疗有进展的地方（参考Loewald，1980Kohut，1877），虽然这时的她看起来只有微弱的力量去保有我的脸的图像。（长时间地停顿]\\n这里实际上匹配了她语句的语调和节奏。毫无疑问，她在下一阶段的电话治疗期间确实是不能乘地铁来看我的。我的脸表现出悲哀的样子，明显表达出她说话时的那个感觉。\\n来访者指的是下一阶段我们的电话治疗，她因住得太远没法乘地铁来看我，电话治疗那天，她曾边沿着街上走，边打着电话给我，就像她正走来面谈似的。母婴互动及成人心理治疗中的主体间形式我点头。我的脸很安静。但是我的脸和我说的话完全（没有改变）地匹配着她说话的语调和节奏。完全的匹配可以形成很特别的共情：这里我只是心无旁地和她呆在一起，完全接受她的表达方式，接受她不连贯的语句。这种互动也解释了困扰调整的一种形式，我所讲过的“加人那沮丧的状态（参考Beebe，2000，2003Cohen&Beebe，2003）。我的头和她的语言跨形态地匹配，可以说明Sterm关于情感协调的概念。\\n', '}', '}']
['{\n    ', 'message', ': ', 'Please provide the input text.', ',\n    ', 'status', ': ', 'failed', '\n}']
['{', 'output', ': ', '治疗师可以通过转换话题或使用特别的提问方式来帮助那些不能表达自己思维的来访者。此外，在来访者报告了初始的自动思维后

In [24]:
write_file("/home/ckqsudo/code2024/EMNLP2024/Chinese_data/cache/gpt_OCR_cache_regular2.json",cn_data)

写入/home/ckqsudo/code2024/EMNLP2024/Chinese_data/cache/gpt_OCR_cache_regular2.json


In [42]:
prev_chat_stage=""
for idx,item in enumerate(cn_data):
    if item['chat_stage']!="":
        if """家庭关系中的权力""" in item['chat_stage']:
            cn_data[idx]['pre_reasoning']="""家庭关系中的权力和性别角色分析这是来访者的第五次治疗。之前，来访者和治疗师确立了她们的治疗关系，她们决定再进行十次后续治疗，这是这十次后续治疗中的第一次。在这个部分我们将看到不同治疗师的不同治疗方式。金凯德深受认知方法的影响，西姆则秉持着精神分析的理论观点，她们二人将通力合作"""+cn_data[idx]['pre_reasoning']
            cn_data[idx]['chat_stage']=""
        print(idx,item['chat_stage'].strip())
        prev_chat_stage=item['chat_stage']
        special_range=[[99,126], [127,156], [157,174]]
        for beg_i,end_i in special_range:
            if idx==beg_i or idx==end_i+1:
                cn_data[idx]["is_same_case"]=0
                break
            if idx>beg_i and idx<=end_i:
                cn_data[idx]["is_same_case"]=1
                break
            else:
                cn_data[idx]["is_same_case"]=-1
    else:
        cn_data[idx]['chat_stage']=prev_chat_stage
        # [99,126] [127,156] [157,174]

0 Regular Session
1 Regular Session
2 Regular Session
3 Regular Session
4 Regular Session
5 Regular Session
6 Regular Session
7 Regular Session
8 1st Session
9 1st Session
10 1st Session
11 Early Session
12 Early Session
13 Early Session
14 Early Session
15 Regular Session
16 Regular Session
17 Regular Session
18 Early Session
19 Early Session
20 1st Session
21 1st Session
22 Regular Session
23 Regular Session
24 Regular Session
25 Regular Session
26 Regular Session
27 Regular Session
28 Regular Session
29 Regular Session
30 Regular Session
31 Early Session
32 Early Session
33 Early Session
34 Regular Session
35 Regular Session
36 Regular Session
37 Regular Session
38 Regular Session
39 Regular Session
40 Regular Session
41 Regular Session
42 Regular Session
43 Early Session
44 Early Session
45 Early Session
46 Early Session
47 Early Session
48 Regular Session
49 Regular Session
50 Regular Session
51 1st Session
52 1st Session
53 Regular Session
54 Regular Session
55 Regular Session
56

In [43]:
write_file("/home/ckqsudo/code2024/EMNLP2024/Chinese_data/cache/gpt_OCR_cache_regular3.json",cn_data)

/home/ckqsudo/code2024/EMNLP2024/Chinese_data/cache/gpt_OCR_cache_regular3.json文件已经存在


In [47]:
from openai import OpenAI
import os
import time
import copy
import json
client = OpenAI()
client.base_url='https://gptnb.keqichen.top/v1'
result=[]

  
token_sum=0
def get_price(total_token:int):
  return f"GPT-4:{token_sum/1000000*10}$ GPT-3:{token_sum/1000000*0.5}$"

for idx,topic in enumerate(cn_data):
    if idx<283:
        continue
    gpt_4="gpt-4-0613"
    gpt_3_5_model="gpt-3.5-turbo"
    if idx==0:
        cn_data[idx]["is_same_case"]=0
    else:
        input_obj={ "chat1":cn_data[idx-1]["topic_dialog"]["dialog"][-2:],
                   "chat2":cn_data[idx]["topic_dialog"]["dialog"][:5]}
        
        example1={"is_same_QA": 1}
        example0={"is_same_QA": 0}
        messages=[
        {"role": "system", "content": f" 这两段对话是否是相关的呢？是否是上下文连贯的呢？例如是的情况返回{json.dumps(example1)},不是的情况返回{json.dumps(example0)}, 请以json形式返回: "},
        {"role": "user","content":json.dumps(input_obj,ensure_ascii=False)}
        ]
        completion = client.chat.completions.create(
            model=gpt_3_5_model,
            messages=messages,
        )
        token_sum+=completion.usage.total_tokens
        choice=completion.choices[0]
        time.sleep(0.5)
        response_str=choice.message.content
        print(idx,token_sum,get_price(token_sum),choice.message.content)
        print(cn_data[idx]["is_same_case"])
        cn_data[idx]["is_same_qa"]=1 if "1" in response_str else 0
        write_file("/home/ckqsudo/code2024/EMNLP2024/Chinese_data/cache/gpt_OCR_cache.json",cn_data,True)
      
      
    # en_data[x]['topic_dialog']['dialog'][y]['emotional label']=choice.message.content


283 310 GPT-4:0.0031$ GPT-3:0.000155$ {"is_same_QA": 0}
-1
写入/home/ckqsudo/code2024/EMNLP2024/Chinese_data/cache/gpt_OCR_cache.json
284 747 GPT-4:0.007470000000000001$ GPT-3:0.0003735$ {"is_same_QA": 0}
-1
写入/home/ckqsudo/code2024/EMNLP2024/Chinese_data/cache/gpt_OCR_cache.json
285 1198 GPT-4:0.011980000000000001$ GPT-3:0.000599$ {"is_same_QA": 0}
-1
写入/home/ckqsudo/code2024/EMNLP2024/Chinese_data/cache/gpt_OCR_cache.json
286 1707 GPT-4:0.01707$ GPT-3:0.0008535$ {"is_same_QA": 0}
-1
写入/home/ckqsudo/code2024/EMNLP2024/Chinese_data/cache/gpt_OCR_cache.json
287 2199 GPT-4:0.02199$ GPT-3:0.0010995$ {"is_same_QA": 0}
-1
写入/home/ckqsudo/code2024/EMNLP2024/Chinese_data/cache/gpt_OCR_cache.json
288 2850 GPT-4:0.0285$ GPT-3:0.001425$ {"is_same_QA": 0}
-1
写入/home/ckqsudo/code2024/EMNLP2024/Chinese_data/cache/gpt_OCR_cache.json
289 3433 GPT-4:0.03433$ GPT-3:0.0017165$ {"is_same_QA": 0}
-1
写入/home/ckqsudo/code2024/EMNLP2024/Chinese_data/cache/gpt_OCR_cache.json
290 3891 GPT-4:0.03891$ GPT-3:0.0019